In [1]:
from glob import glob
import pandas as pd
import numpy as np
from os import makedirs
from os.path import dirname
from os.path import basename, splitext
import time
from contextlib import contextmanager

In [2]:
def check_merge(df, source):
    type_doc = ["СчФктр", "УпдДоп", "УпдСчфДоп", "ЭДОНакл"]
    
    df.insert(8, "Номер счет-фактуры", value="")
    df.insert(9, "Сумма счет-фактуры", value="") 
    df["Дата счет-фактуры"] = ""
    df["Сравнение дат"] = ""
    # Оставляем только первые строки для каждого номера
    filtered_source = (
        source[source['Тип документа'].isin(type_doc)]
        .drop_duplicates(subset='Номер', keep='first')
    )
    filtered_source = filtered_source.add_suffix('_source')
    merged = pd.merge(df,
        filtered_source,
        left_on='Номер накладной',
        right_on='Номер_source',
        how='left'
    )
    merged['Дата.1_source'] = pd.to_datetime(
        merged['Дата.1_source'], 
        dayfirst=True,
        format='%d.%m.%y'
    ).dt.strftime('%d.%m.%Y')
    
    merged['Дата накладной'] = pd.to_datetime(
        merged['Дата накладной'], 
        dayfirst=True,
        format='%d.%m.%Y'
    ).dt.strftime('%d.%m.%Y')

    merged['Номер счет-фактуры'] = merged['Номер_source'].mask(merged['Поставщик'] == 'ЕАПТЕКА', merged['Номер_source'] + '/15')
    merged['Сумма счет-фактуры'] = merged['Сумма_source']
    merged['Дата счет-фактуры'] =  merged['Дата.1_source']

    merged['Сравнение дат'] = np.where((merged['Дата накладной'] != merged['Дата счет-фактуры']) & (merged['Дата счет-фактуры'].notna()) , 'Не совпадает!', '')

    return merged[df.columns]

In [3]:
@contextmanager
def timer():
    start_time = time.time()
    try:
        yield
    finally:
        end_time = time.time()
        print(f"Время выполнения: {round(end_time - start_time, 2)} секунд")

with timer():
    path = 'C:/Users/Pala/Documents/jupyter/practise_pandas'
    current_date = pd.Timestamp('now').date().strftime('%d-%m-%Y')
    folder_source = '/Входящие'
    folder_apteka = '/Аптеки/csv/correct'
    
    lst_source_files = glob(path + folder_source + '/*.csv')
    lst_apteka_files = glob(path + folder_apteka + '/*.csv')
    lst_source_dfs = [pd.read_csv(item, encoding='cp1251', engine='python', sep=';' ) for item in lst_source_files]
    source_df = pd.concat(lst_source_dfs, ignore_index=True)
        
    for filepath in lst_apteka_files:
        df = pd.read_csv(filepath, encoding='cp1251', engine='python', sep=';' )
        res = check_merge(df, source_df)
        # Создаем путь с папками
        filename = splitext(basename(filepath))[0]
        res_file_path = f'{path}/Результат/{current_date}/{filename} - результат.xlsx'
        # Создаем папки если их нет
        makedirs(dirname(res_file_path), exist_ok=True)
        # Сохраняем файл
        res.to_excel(res_file_path, index=False)
        print(f'файл "{filename}" обработан')

файл "366" обработан
файл "А123" обработан
Время выполнения: 10.53 секунд
